In [1]:
import pandas as pd

sdcard = "./dados-para-analise/sdcard-toplot-filtered.csv"
smarthub_csv = "./dados-para-analise/smarthub-toplot.csv"
df_sdcard = pd.read_csv(sdcard, sep=";", decimal=".", encoding="latin1")
df_smarthub = pd.read_csv(smarthub_csv, sep=";", decimal=".", encoding="latin1")
print(df_smarthub.columns.to_list())
print(df_sdcard.columns.to_list())

['timestamp', 'T_MF4_IN', 'T_MF4_OUT', 'T_MM2_IN', 'T_MM2_OUT', 'T_BOILER100', 'T_BOILER200', 'T_IN', 'Sinal_Vazao1']
['T_BOILER100', 'T_BOILER200', 'T_IN', 'T_MF4_IN', 'T_MF4_OUT', 'T_MM2_IN', 'T_MM2_OUT', 'timestamp']


# Análise de confiança da plataforma

Teoricamene em um período de 24h são feitas 86400 leituras no equipamento de campo (1 leitura por segundo). Essas leituras são salvas em cartão de memória. No período de 24h selecionado foram feitas 79101 salvamentos na memória do equipamento. A plataforma fez o registro de 20663 telemetrias. 


In [2]:
"""Diante da premissa acima, o código abaixo dita a porcentagem de leituras que foram salvas na plataforma"""

n_dados_smarthub = df_smarthub["timestamp"].count()
n_dados_sdcard = df_sdcard["timestamp"].count()
print("Quantidade de dados do smarthub: ", n_dados_smarthub)
print("Quantidade de dados do sdcard: ", n_dados_sdcard)
porcentagem = (n_dados_smarthub / n_dados_sdcard) * 100
print(
    "Porcentagem de dados do smarthub em relação ao sdcard: ",
    porcentagem.__round__(2),
    "%",
)

Quantidade de dados do smarthub:  20663
Quantidade de dados do sdcard:  79101
Porcentagem de dados do smarthub em relação ao sdcard:  26.12 %


# Análise de verossimilhança

Para comparar a fidelidade dos dados que foram guardados no banco da plataforma. Iremos comparar os dados com os mesmos timestamps de ambos os sets. Abaixo serão plotados os gráficos de cada variável sendo avaliada. Além disso será impresso um erro médio.

In [3]:
def buscar_indice_df(df, coluna, valor):
    """
    Busca o índice de uma linha em um DataFrame com base no valor de uma coluna específica.

    Args:
        df (pandas.DataFrame): O DataFrame onde a busca será realizada.
        coluna (str): O nome da coluna onde o valor será procurado.
        valor (Any): O valor a ser buscado na coluna especificada.

    Returns:
        int: O índice da primeira ocorrência do valor na coluna, ou -1 se o valor não for encontrado.

    Exemplo:
        >>> import pandas as pd
        >>> data = {'coluna1': [1, 2, 3], 'coluna2': ['a', 'b', 'c']}
        >>> df = pd.DataFrame(data)
        >>> buscar_indice_df(df, 'coluna1', 2)
        1
        >>> buscar_indice_df(df, 'coluna2', 'x')
        -1
    """
    try:
        return df[df[coluna] == valor].index[0]
    except IndexError:
        return -1  # Retorna -1 se o valor não for encontrado


# 1. Certifique-se de que as colunas de timestamp são do mesmo tipo
df_sdcard["timestamp"] = pd.to_datetime(df_sdcard["timestamp"])
df_smarthub["timestamp"] = pd.to_datetime(df_smarthub["timestamp"])

# 2. Crie o DataFrame final com todas as colunas necessárias
colunas = [
    "timeStamp",
    "T_MF4_IN_sd",
    "T_MF4_IN_sh",
    "T_MF4_OUT_sd",
    "T_MF4_OUT_sh",
    "T_MM2_IN_sd",
    "T_MM2_IN_sh",
    "T_MM2_OUT_sd",
    "T_MM2_OUT_sh",
    "T_BOILER100_sd",
    "T_BOILER100_sh",
    "T_BOILER200_sd",
    "T_BOILER200_sh",
    "T_IN_sd",
    "T_IN_sh",
]

df_final = pd.DataFrame(columns=colunas)


# 3. Função otimizada para buscar índices
def buscar_indice_seguro(df, coluna, valor):
    matches = df[df[coluna] == valor]
    return matches.index[0] if not matches.empty else -1


# 4. Processamento principal
for i, smarthub_row in df_smarthub.iterrows():
    timestamp = smarthub_row["timestamp"]
    index_sdcard = buscar_indice_seguro(df_sdcard, "timestamp", timestamp)

    if index_sdcard != -1:
        sdcard_row = df_sdcard.loc[index_sdcard]

        new_row = {
            "timeStamp": timestamp,
            # Dados do SD Card (sufixo _sd)
            "T_MF4_IN_sd": sdcard_row.get("T_MF4_IN"),
            "T_MF4_OUT_sd": sdcard_row.get("T_MF4_OUT"),
            "T_MM2_IN_sd": sdcard_row.get("T_MM2_IN"),
            "T_MM2_OUT_sd": sdcard_row.get("T_MM2_OUT"),
            "T_BOILER100_sd": sdcard_row.get("T_BOILER100"),
            "T_BOILER200_sd": sdcard_row.get("T_BOILER200"),
            "T_IN_sd": sdcard_row.get("T_IN"),
            # Dados do SmartHub (sufixo _sh)
            "T_MF4_IN_sh": smarthub_row.get("T_MF4_IN"),
            "T_MF4_OUT_sh": smarthub_row.get("T_MF4_OUT"),
            "T_MM2_IN_sh": smarthub_row.get("T_MM2_IN"),
            "T_MM2_OUT_sh": smarthub_row.get("T_MM2_OUT"),
            "T_BOILER100_sh": smarthub_row.get("T_BOILER100"),
            "T_BOILER200_sh": smarthub_row.get("T_BOILER200"),
            "T_IN_sh": smarthub_row.get("T_IN")
        }

        df_final.loc[len(df_final)] = new_row

# 5. Opcional: Resetar o índice se necessário
df_final.reset_index(drop=True, inplace=True)
# print(df_final.head())
dir_salvo = "./dados-para-analise/df_final.csv"
df_final.to_csv(dir_salvo, sep=";", decimal=".", index=False, encoding="latin1")
print("O dataframe final tem ", len(df_final), " linhas e ", len(df_final.columns), " colunas.")
print("O dataframe final foi salvo em ", dir_salvo)

O dataframe final tem  20663  linhas e  15  colunas.
O dataframe final foi salvo em  ./dados-para-analise/df_final.csv


In [15]:
def erro_medio(df, coluna1, coluna2):
    """
    Calcula o erro médio entre duas colunas de um DataFrame.

    Args:
        df (pandas.DataFrame): O DataFrame contendo as colunas.
        coluna1 (str): Nome da primeira coluna.
        coluna2 (str): Nome da segunda coluna.

    Returns:
        float: O erro médio entre as duas colunas.
    """
    return (df[coluna1] - df[coluna2]).abs().mean()

def printa_erros_medios():
    print(
        "Erro médio entre colunas T_MF4_IN: ",
        erro_medio(df_final, "T_MF4_IN_sd", "T_MF4_IN_sh").__round__(4),
    )
    print(
        "Erro médio entre colunas T_MF4_OUT: ",
        erro_medio(df_final, "T_MF4_OUT_sd", "T_MF4_OUT_sh").__round__(4),
    )
    print(
        "Erro médio entre colunas T_MM2_IN: ",
        erro_medio(df_final, "T_MM2_IN_sd", "T_MM2_IN_sh").__round__(4),
    )
    print(
        "Erro médio entre colunas T_MM2_OUT: ",
        erro_medio(df_final, "T_MM2_OUT_sd", "T_MM2_OUT_sh").__round__(4),
    )
    print(
        "Erro médio entre colunas T_BOILER100: ",
        erro_medio(df_final, "T_BOILER100_sd", "T_BOILER100_sh").__round__(4),
    )
    print(
        "Erro médio entre colunas T_BOILER200: ",
        erro_medio(df_final, "T_BOILER200_sd", "T_BOILER200_sh").__round__(4),
    )
    print(
        "Erro médio entre colunas T_IN: ",
        erro_medio(df_final, "T_IN_sd", "T_IN_sh").__round__(4),
    )


printa_erros_medios()

Erro médio entre colunas T_MF4_IN:  3.7542
Erro médio entre colunas T_MF4_OUT:  2.4812
Erro médio entre colunas T_MM2_IN:  1.6806
Erro médio entre colunas T_MM2_OUT:  1.651
Erro médio entre colunas T_BOILER100:  0.0066
Erro médio entre colunas T_BOILER200:  0.0065
Erro médio entre colunas T_IN:  0.0
